## Основы pytorch

Для выполнения ДЗ создайте приватный репозиторий и добавьте `https://github.com/norsage` в collaborators (Settings -> Collaborators -> Add people)

In [1]:
import torch

#### 1. Операции над тензорами (1 балл)

##### 1.1 Среднее значение по столбцам

In [2]:
torch.manual_seed(42)
x = torch.randint(10, size=(2, 3)).float()
x

tensor([[2., 7., 6.],
        [4., 6., 5.]])

In [3]:
mean_by_row = x.mean(dim=0)
assert torch.allclose(
    mean_by_row, _expected := torch.tensor([3.0, 6.5, 5.5])
), f"{mean_by_row} != {_expected}"

##### 1.2. Взвешенное среднее
В тензоре `w` находятся ненормализованные веса для расчёта взвешенных средних тензора `x` по строкам.

Найдите эти взвешенные средние, получая нормализованные веса с помощью функции `torch.softmax` (или метода `.softmax`)

In [4]:
torch.manual_seed(42)
x = torch.randint(10, size=(2, 3)).float()
w = torch.randint(10, size=(2, 3)).float()
print(x)
print(w)

tensor([[2., 7., 6.],
        [4., 6., 5.]])
tensor([[0., 4., 0.],
        [3., 8., 4.]])


In [5]:
w_avg = torch.sum(w.softmax(dim = 1) * x, dim=1)
assert torch.allclose(
    w_avg, _expected := torch.tensor([6.8940, 5.9690])
), f"{w_avg} != {_expected}"

##### 1.3. Умножение матриц на векторы

В тензоре `m` - две матрицы, нужно сделать тензор, в котором i-й элемент - результат умножения матрицы `m[i]` на вектор `x[i]`.

Это можно было бы сделать так: `torch.stack([m[i] @ x[i] for i in len(m)])`.

Попробуйте найти решение без цикла.

In [6]:
torch.manual_seed(42)
x = torch.randint(10, size=(2, 3)).float()
m = torch.randint(10, size=(2, 3, 3)).float()
print(m)
print(x)

tensor([[[0., 4., 0.],
         [3., 8., 4.],
         [0., 4., 1.]],

        [[2., 5., 5.],
         [7., 6., 9.],
         [6., 3., 1.]]])
tensor([[2., 7., 6.],
        [4., 6., 5.]])


In [7]:
matmul = (m @ x.reshape(2, 3, 1)).reshape(2, 3)
assert torch.allclose(
    matmul, _expected := torch.tensor([[28.0, 86.0, 34.0], [63.0, 109.0, 47.0]])
), f"{matmul} != {_expected}"

##### 1.4. Матрица попарных расстояний

Даны две матрицы `x` и `y`, нужно получить матрицу `d`, где `d[i, j]` - евклидово расстояние между векторами `x[i]` и `y[j]`.

Подсказка 1: воспользуйтесь broadcasting и добавлением размерностей в исходные тензоры.

Подсказка 2: можно не считать евклидово расстояние вручную, есть функция `torch.linalg.norm`



In [8]:
torch.manual_seed(42)
x = torch.randint(10, size=(2, 3)).float()
y = torch.randint(10, size=(3, 3)).float()
print(x)
print(y)

tensor([[2., 7., 6.],
        [4., 6., 5.]])
tensor([[0., 4., 0.],
        [3., 8., 4.],
        [0., 4., 1.]])


In [9]:
pdist = torch.linalg.norm(x.reshape(2, 1, 3) - y, dim = 2)
assert torch.allclose(
    pdist,
    _expected := torch.tensor([[7.0000, 2.4495, 6.1644], [6.7082, 2.4495, 6.0000]]),
), f"{pdist} != {_expected}"

#### 2. Функция Power (1 балл)
Используя сложение и умножение, реализуйте возведение в целочисленную степень FloatTensor как функцию autograd (т.е. наследника `torch.autograd.Function`)

In [10]:
class Power(torch.autograd.Function):
    @staticmethod
    def forward(tensor, p):
        result = torch.ones_like(tensor)
        if p == 0:
            return result
        elif p > 0:
            for _ in range(p):
                result = result * tensor
        else:
            for _ in range(-p):
                result = result / tensor

        return result

    @staticmethod
    def setup_context(ctx, inputs, output):
        # ctx is a context object that can be used to stash information
        # for backward computation
        tensor, p = inputs
        ctx.p = p
        ctx.tensor = tensor

    @staticmethod
    def backward(ctx, grad_output):
        tensor, p = ctx.tensor, ctx.p
        grad_tensor = torch.ones_like(tensor)

        if p > 0:
            for i in range(p - 1):
                grad_tensor = grad_tensor * tensor
            return grad_output * grad_tensor * p, None
        elif p < 0:
            for i in range(-p + 1):
                grad_tensor = grad_tensor / tensor
            return grad_output * grad_tensor * p, None
        else:
            return grad_output.zero_(), None

In [11]:
x = torch.tensor(3.0, requires_grad=True)
y = Power.apply(x, 2)
y.backward()

a = torch.tensor(3.0, requires_grad=True)
b = a ** 2
b.backward()

assert torch.all(x.grad == a.grad)

x = torch.tensor(3.0, requires_grad=True)
y = Power.apply(x, -2)
y.backward()

a = torch.tensor(3.0, requires_grad=True)
b = a ** -2
b.backward()

assert torch.all(x.grad == a.grad)

x = torch.tensor(3.0, requires_grad=True)
y = Power.apply(x, 0)
y.backward()

a = torch.tensor(3.0, requires_grad=True)
b = a ** 0
b.backward()

assert torch.all(x.grad == a.grad)

In [12]:
assert torch.all(Power.apply(torch.tensor([1, 2, 3]), 0) == torch.tensor([1, 1, 1]))
assert torch.all(Power.apply(torch.tensor([1, 2, 3]), 2) == torch.tensor([1, 4, 9]))

#### 3. Многочлен (3 балла)
Найдите корень (он один) заданного полинома (очень хорошего!) с точностью до пяти знаков после запятой:
1. Используя бинарный поиск https://en.wikipedia.org/wiki/Binary_search_algorithm
2. Используя метод Ньютона https://en.wikipedia.org/wiki/Newton%27s_method
   
   Задаётся начальное приближение вблизи предположительного корня, после чего строится касательная к графику исследуемой функции в точке приближения, для которой находится пересечение с осью абсцисс. Эта точка берётся в качестве следующего приближения. И так далее, пока не будет достигнута необходимая точность.
   
   (hint: для вычисления производных используйте метод `backward()`)
   
   $x_{n+1} = x_{n} - \frac{f(x_n)}{f'(x_n)}$

Сравните скорость методов с помощью `%%timeit`, т.е. оцените, какой из них найдёт ответ быстрее

In [13]:
from typing import Callable

Polynomial = Callable[[torch.FloatTensor], torch.FloatTensor]


def poly(x: torch.FloatTensor) -> torch.FloatTensor:
    return x**7 + 5 * x**3 + 17 * x - 9

In [14]:
def bin_search_find_zero(poly: Polynomial) -> torch.FloatTensor:
    """Функция для бинарного поиска"""
    tol = torch.tensor(10**-5)

    low = torch.tensor(-10)
    high = torch.tensor(10)

    while high - low > tol:
        mid = (low + high) / 2
        if poly(mid) == 0:
            return mid
        elif poly(mid) > 0:
            high = mid
        else:
            low = mid

    return (low + high) / 2

In [15]:
def newton_find_zero(poly: Polynomial) -> torch.FloatTensor:
    """Функция для метода Ньютона"""

    # первое приближение (не забываем про то, что понадобится градиент!)
    x = torch.tensor(0.0, requires_grad=True).float()

    # останавливаемся, если значение функции достаточно близко к нулю
    tol = 10**-5

    # значение
    val = poly(x)

    # цикл обновления
    while abs(val) > tol:  # когда останавливаемся?
        val.backward()

        with torch.no_grad():
            x -= val / x.grad

        x.grad = None 
        
        val = poly(x)
        
    return x

In [16]:
x = bin_search_find_zero(poly)
print(x)
print(poly(x))

tensor(0.4936)
tensor(-9.5367e-07)


In [17]:
x = newton_find_zero(poly)
print(x)
print(poly(x))

tensor(0.4936, requires_grad=True)
tensor(1.9073e-06, grad_fn=<SubBackward0>)


In [18]:
%%timeit
x = newton_find_zero(poly)

258 μs ± 3.99 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
%%timeit
x = bin_search_find_zero(poly)

916 μs ± 26.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
